# EfficientNet Simple baseline implementation

Hello Readers,

This is very simple baseline implementation using EfficientNet and very good source for beginners to start with the competition.
Please good through the notebook and let me know your thoughts and feedbacks in the comment section.

Also, if you like the notebook do remember to upvote please :) !!

In [ ]:
# Importing the libraries
import os
import glob
import shutil
import json
import keras
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import EfficientNetB4
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import albumentations

# Defining the directories
work_dir = '../input/cassava-leaf-disease-classification/'
os.listdir(work_dir) 
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'

In [ ]:
# Dataframe for train.csv
data = pd.read_csv(work_dir + 'train.csv')

In [ ]:

f = open(work_dir + 'label_num_to_disease_map.json')
real_labels = json.load(f)
real_labels = {int(k):v for k,v in real_labels.items()}

# Defining the working dataset
data['class_name'] = data.label.map(real_labels)

train,val = train_test_split(data, test_size = 0.1, random_state = 42, stratify = data['class_name'])

IMG_SIZE = 380
size = (IMG_SIZE,IMG_SIZE)
n_CLASS = 5
BATCH_SIZE = 20

datagen_train = ImageDataGenerator(
                    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
                    rotation_range = 40,
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest')

datagen_val = ImageDataGenerator(
                    preprocessing_function = tf.keras.applications.efficientnet.preprocess_input,
                    )

train_data = datagen_train.flow_from_dataframe(train,
                             directory = train_path,
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = size,
                             #color_mode="rgb",
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = BATCH_SIZE)

val_data = datagen_val.flow_from_dataframe(val,
                             directory = train_path,
                             seed=42,
                             x_col = 'image_id',
                             y_col = 'class_name',
                             target_size = size,
                             #color_mode="rgb",
                             class_mode = 'categorical',
                             interpolation = 'nearest',
                             shuffle = True,
                             batch_size = BATCH_SIZE)


In [ ]:
def create_model():
    
    model = Sequential()    
    model.add(EfficientNetB4(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False, 
                             weights = '../input/tfkerasefficientnetimagenetnotop/efficientnetb4_notop.h5', 
                             drop_connect_rate=0.4))
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(512, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(n_CLASS, activation = 'softmax'))
    
    return model

leaf_model = create_model()
leaf_model.summary()

In [ ]:
EPOCHS = 11
SST = train_data.n//train_data.batch_size
SSV = val_data.n//val_data.batch_size

In [ ]:
def model_fitter():
        
    leaf_model = create_model()
        
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False, label_smoothing=0.05,name='categorical_crossentropy' )
    
    leaf_model.compile(optimizer = Adam(learning_rate = 1e-3), loss = loss, metrics = ['categorical_accuracy'])
    
    es = EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True, verbose=1)
    
    checkpoint_cb = ModelCheckpoint("Cassava_best_model.h5", save_best_only=True, monitor = 'val_loss', mode='min')
    
    reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, patience = 2, min_lr = 1e-6, mode = 'min', verbose = 1)
    
    history = leaf_model.fit(train_data, validation_data = val_data, epochs= EPOCHS, batch_size = BATCH_SIZE,
                             steps_per_epoch = SST,
                             validation_steps = SSV,
                             callbacks=[es, checkpoint_cb, reduce_lr])
    
    leaf_model.save('Cassava_model'+'.h5')  
    
    return history


In [ ]:
results = model_fitter()

In [ ]:
# EVALUATION METRIC

print('Training Accuracy: ', max(results.history['categorical_accuracy']))
print('Validation Accuracy: ', max(results.history['val_categorical_accuracy']))

In [ ]:
import keras

final_model = keras.models.load_model('Cassava_best_model.h5')

In [ ]:
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'
test_images = os.listdir(TEST_DIR)
predictions = []

for image in test_images:
    img = Image.open(TEST_DIR + image)
    img = img.resize(size)
    img = np.expand_dims(img, axis=0)
    predictions.extend(final_model.predict(img).argmax(axis = 1))

In [ ]:
predictions

In [ ]:
sub = pd.DataFrame({'image_id': test_images, 'label': predictions})
display(sub)
sub.to_csv('submission.csv', index = False)